# Generate only the `processInformation` part of the Schema and JSON Instance

In [1]:
# Produce Python module from SharedDefinitions using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator

sd_yaml_schema = "../linkml/data/yaml/linkml_shared_definitions.yaml"
sd_python_file = "../linkml/data/py/linkml_shared_definitions.py"

# Define an import map so that types like "String", "Boolean", etc.
# are resolved to their LinkML runtime equivalents.
import_map = {
    "Boolean": "linkml_runtime.linkml_model.types.Boolean",
    "String": "linkml_runtime.linkml_model.types.String",
    "Integer": "linkml_runtime.linkml_model.types.Integer",
    "Float": "linkml_runtime.linkml_model.types.Float",
    "dateTime": "linkml_runtime.linkml_model.types.dateTime",
    # "Version": "linkml_runtime.linkml_model.types.Version", # Not in LinkML runtime
    # "anyURI": "linkml_runtime.linkml_model.types.anyURI"  # Not in LinkML runtime
}

generator = PythonGenerator(
    schema=sd_yaml_schema,
    gen_slots=True,
    gen_classvars=True,
    mergeimports=True,
    metadata=True,
    genmeta=False,
    importmap=import_map,
)


python_code = generator.serialize()


with open(sd_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {sd_python_file}.")

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Python model generated and written to ../linkml/data/py/linkml_shared_definitions.py.


In [2]:
# Produce Python module from YAML schema using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator


pi_yaml_schema = "../linkml/data/yaml/linkml_processInformation_schema.yaml"
pi_python_file = "../linkml/data/py/linkml_processInformation_schema.py"


# Define an import map so that types like "String", "Boolean", etc.
# are resolved to their LinkML runtime equivalents.
import_map = {
    "Boolean": "linkml_runtime.linkml_model.types.Boolean",
    "String": "linkml_runtime.linkml_model.types.String",
    "Integer": "linkml_runtime.linkml_model.types.Integer",
    "Float": "linkml_runtime.linkml_model.types.Float",
    "dateTime": "linkml_runtime.linkml_model.types.dateTime",
    # "Version": "linkml_runtime.linkml_model.types.Version", # Not in LinkML runtime
    # "anyURI": "linkml_runtime.linkml_model.types.anyURI"  # Not in LinkML runtime
}

generator = PythonGenerator(
    schema=pi_yaml_schema,
    gen_slots=True,
    gen_classvars=True,
    mergeimports=True,
    metadata=True,
    genmeta=False,
    importmap=import_map,
)


python_code = generator.serialize()

with open(pi_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {pi_python_file}.")

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Python model generated and written to ../linkml/data/py/linkml_processInformation_schema.py.


In [3]:
# Instance Generation and Loading processInformation EPD YAML

import json
import yaml
from collections import OrderedDict
from data.py.linkml_processInformation_schema import ProcessInformation
from linkml.validator import Validator
from linkml_runtime.loaders.yaml_loader import YAMLLoader


def odict_to_dict(obj):
    """
    Recursively convert an OrderedDict (or list/dict containing OrderedDicts) to a standard dict.

    Parameters:
        obj: An OrderedDict, list, or dict potentially containing OrderedDicts.

    Returns:
        A new structure where all OrderedDict instances are replaced with standard dicts.
    """
    if isinstance(obj, OrderedDict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [odict_to_dict(item) for item in obj]
    elif isinstance(obj, dict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    else:
        return obj


def build_minimal_epd_structure(data):
    """
    Build a minimal nested dictionary according to our minimal schema.
    This function assumes that the JSON instance already contains a "processInformation" key.

    Parameters:
        data (dict): A dictionary loaded from a JSON file.

    Returns:
        dict: A dictionary structured with a top-level 'processInformation' key.
    """
    instance_dict = {"processInformation": data.get("processInformation", {})}
    return instance_dict


def process_instance(input_path, schema_path, output_yaml_path):
    """
    Process an instance by loading a minimal JSON instance, converting it, dumping it to YAML,
    validating it against a schema, and loading it with YAMLLoader.

    Steps:
      1. Load a minimal JSON instance containing only the processInformation section.
      2. Convert OrderedDicts to dicts.
      3. Wrap the data in a top-level ProcessInformation structure.
      4. Dump the resulting instance to a YAML file.
      5. Validate the instance using a schema validator.
      6. Load the YAML instance with YAMLLoader.

    Parameters:
        input_path (str): Path to the input JSON file.
        schema_path (str): Path to the YAML schema file.
        output_yaml_path (str): Path where the output YAML file will be written.

    Returns:
        None
    
    Raises:
        Exception: If an error occurs during validation or instance loading.
    """
    # Load the minimal JSON instance (containing only processInformation)
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    data = odict_to_dict(data)

    # Build the nested instance structure.
    instance_data = build_minimal_epd_structure(data)

    # Dump to YAML
    with open(output_yaml_path, "w", encoding="utf-8") as out:
        yaml.dump(instance_data, out, sort_keys=False, allow_unicode=True)
    print(f"processInformation EPD YAML instance written to: {output_yaml_path}")

    # Validate using the Validator
    try:
        validator = Validator(schema_path, strict=False)
        report = validator.validate(instance_data, "ProcessInformation")
        if report.results:
            print("Validation errors:")
            for result in report.results:
                print(result.message)
        else:
            print("The ProcessInformation instance is valid!")
    except Exception as e:
        print("Exception during validation:")
        print(e)
        raise

    # Load the YAML instance using YAMLLoader
    try:
        loader = YAMLLoader()
        # If the instance is wrapped in a "processInformation" key, unwrap it.
        if "processInformation" in instance_data:
            instance_data = instance_data["processInformation"]
        loader.load(instance_data, target_class=ProcessInformation)
        print("Instance loaded successfully!")
    except Exception as e:
        print("Exception during instance loading via YAMLLoader:")
        print(e)
        raise


# Usage
input_file = "../linkml/data/json/instance_processInformation.json"
schema_file = "../linkml/data/yaml/linkml_processInformation_schema.yaml"
output_yaml = "../linkml/data/yaml/linkml_processInformation_instance.yaml"

process_instance(input_file, schema_file, output_yaml)

processInformation EPD YAML instance written to: ../linkml/data/yaml/linkml_processInformation_instance.yaml
The ProcessInformation instance is valid!
Instance loaded successfully!


In [4]:
# Loading and RDF (JSON-LD, Turtle) Generation processInformation EPD YAML Instance

import yaml
from linkml_runtime.dumpers import RDFLibDumper
from linkml_runtime.loaders import YAMLLoader
from linkml_runtime.utils.schemaview import SchemaView
from data.py.linkml_processInformation_schema import ProcessInformation


def generate_rdf(schema_path: str, instance_path: str, output_jsonld_path: str, output_turtle_path: str) -> None:
    """
    Load a validated LinkML YAML instance, create a SchemaView, and dump the instance to RDF in JSON-LD and Turtle formats.

    Steps:
        1. Load the instance from a YAML file.
        2. Create a SchemaView from the schema file.
        3. Generate RDF in JSON-LD and Turtle formats.
        4. Write the RDF to files.
    
    Parameters:
        schema_path (str): Path to the LinkML YAML schema file.
        instance_path (str): Path to the validated LinkML YAML instance file.
        output_jsonld_path (str): Path where the output JSON-LD file will be written.
        output_turtle_path (str): Path where the output Turtle file will be written.
    
    Returns:
        None

    Raises:
        Exception: If an error occurs during instance loading, SchemaView creation, or RDF generation.
    """
    # Load instance from file
    try:
        with open(instance_path, "r", encoding="utf-8") as f:
            loaded_data = yaml.safe_load(f)  # This is now a Python dict
        # Extract "processInformation"
        if "processInformation" in loaded_data:
            loaded_data = loaded_data["processInformation"]
        # Feed loaded_data dictionary into the loader
        instance_obj = YAMLLoader().load(loaded_data, target_class=ProcessInformation)
        print(f"Instance loaded from: {instance_path}")
    except Exception as e:
        print("Exception during instance loading:")
        print(e)
        raise

    # Create SchemaView
    try:
        sv = SchemaView(schema_path)
        print("SchemaView created successfully.")
    except Exception as e:
        print("Exception during SchemaView creation:")
        print(e)
        raise

    # Generate and write RDF (JSON-LD, Turtle)
    try:
        dumper = RDFLibDumper()
        # JSON‑LD format
        rdf_jsonld = dumper.dumps(instance_obj, schemaview=sv, fmt="json-ld")
        with open(output_jsonld_path, "w", encoding="utf-8") as rdf_file:
            rdf_file.write(rdf_jsonld)
        print(f"RDF (JSON‑LD) successfully written to: {output_jsonld_path}")
        # Turtle format
        rdf_turtle = dumper.dumps(instance_obj, schemaview=sv, fmt="turtle")
        with open(output_turtle_path, "w", encoding="utf-8") as ttl_file:
            ttl_file.write(rdf_turtle)
        print(f"RDF (Turtle) successfully written to: {output_turtle_path}")
    except Exception as e:
        print("Exception during RDF generation:")
        print(e)
        raise


# Example usage:
schema_path = "../linkml/data/yaml/linkml_processInformation_schema.yaml"
instance_path = "../linkml/data/yaml/linkml_processInformation_instance.yaml"
output_jsonld_path = "../linkml/data/rdf/linkml_processInformation_instance.jsonld"
output_turtle_path = "../linkml/data/rdf/linkml_processInformation_instance.ttl"

generate_rdf(schema_path, instance_path, output_jsonld_path, output_turtle_path)

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Instance loaded from: ../linkml/data/yaml/linkml_processInformation_instance.yaml
SchemaView created successfully.
RDF (JSON‑LD) successfully written to: ../linkml/data/rdf/linkml_processInformation_instance.jsonld
RDF (Turtle) successfully written to: ../linkml/data/rdf/linkml_processInformation_instance.ttl


# Generate only the `modellingAndValidation` part of the Schema and JSON Instance

In [1]:
# Produce Python module from YAML schema using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator

sd_yaml_schema = "../linkml/data/yaml/linkml_shared_definitions.yaml"
sd_python_file = "../linkml/data/py/linkml_shared_definitions.py"

# Define an import map so that types like "String", "Boolean", etc.
# are resolved to their LinkML runtime equivalents.
import_map = {
    "Boolean": "linkml_runtime.linkml_model.types.Boolean",
    "String": "linkml_runtime.linkml_model.types.String",
    "Integer": "linkml_runtime.linkml_model.types.Integer",
    "Float": "linkml_runtime.linkml_model.types.Float",
    "dateTime": "linkml_runtime.linkml_model.types.dateTime",
    # "Version": "linkml_runtime.linkml_model.types.Version", # Not in LinkML runtime
    # "anyURI": "linkml_runtime.linkml_model.types.anyURI"  # Not in LinkML runtime
}

generator = PythonGenerator(
    schema=sd_yaml_schema,
    gen_slots=True,
    gen_classvars=True,
    mergeimports=True,
    metadata=True,
    genmeta=False,
    importmap=import_map,
)


python_code = generator.serialize()


with open(sd_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {sd_python_file}.")

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Python model generated and written to ../linkml/data/py/linkml_shared_definitions.py.


In [2]:
# Produce Python module from YAML schema using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator

mav_yaml_schema = "../linkml/data/yaml/linkml_modellingAndValidation_schema.yaml"
mav_python_file = "../linkml/data/py/linkml_modellingAndValidation_schema.py"

# Define an import map so that types like "String", "Boolean", etc.
# are resolved to their LinkML runtime equivalents.
import_map = {
    "Boolean": "linkml_runtime.linkml_model.types.Boolean",
    "String": "linkml_runtime.linkml_model.types.String",
    "Integer": "linkml_runtime.linkml_model.types.Integer",
    "Float": "linkml_runtime.linkml_model.types.Float",
    "dateTime": "linkml_runtime.linkml_model.types.dateTime",
    # "Version": "linkml_runtime.linkml_model.types.Version", # Not in LinkML runtime
    # "anyURI": "linkml_runtime.linkml_model.types.anyURI"  # Not in LinkML runtime
}

generator = PythonGenerator(
    schema=mav_yaml_schema,
    gen_slots=True,
    gen_classvars=True,
    mergeimports=True,
    metadata=True,
    genmeta=False,
    importmap=import_map,
)


python_code = generator.serialize()

with open(mav_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {mav_python_file}.")

Overlapping slot and class names: LCIMethodAndAllocation
	
	
c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Python model generated and written to ../linkml/data/py/linkml_modellingAndValidation_schema.py.


In [3]:
# Instance Generation and Loading modellingAndValidation EPD YAML

import json
import yaml
from collections import OrderedDict
from data.py.linkml_modellingAndValidation_schema import ModellingAndValidation
from linkml.validator import Validator
from linkml_runtime.loaders.yaml_loader import YAMLLoader


def odict_to_dict(obj):
    """
    Recursively convert an OrderedDict (or list/dict containing OrderedDicts) to a standard dict.

    Parameters:
        obj: An OrderedDict, list, or dict potentially containing OrderedDicts.

    Returns:
        A new structure where all OrderedDict instances are replaced with standard dicts.
    """
    if isinstance(obj, OrderedDict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [odict_to_dict(item) for item in obj]
    elif isinstance(obj, dict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    else:
        return obj


def build_minimal_epd_structure(data):
    """
    Build a minimal nested dictionary according to our minimal schema.
    This function assumes that the JSON instance already contains a "modellingAndValidation" key.

    Parameters:
        data (dict): A dictionary loaded from a JSON file.

    Returns:
        dict: A dictionary structured with a top-level 'modellingAndValidation' key.
    """
    instance_dict = {"modellingAndValidation": data.get("modellingAndValidation", {})}
    return instance_dict


def process_instance(input_path, schema_path, output_yaml_path):
    """
    Process an instance by loading a minimal JSON instance, converting it, dumping it to YAML,
    validating it against a schema, and loading it with YAMLLoader.

    Steps:
      1. Load a minimal JSON instance containing only the modellingAndValidation section.
      2. Convert OrderedDicts to dicts.
      3. Wrap the data in a top-level ModellingAndValidation structure.
      4. Dump the resulting instance to a YAML file.
      5. Validate the instance using a schema validator.
      6. Load the YAML instance with YAMLLoader.

    Parameters:
        input_path (str): Path to the input JSON file.
        schema_path (str): Path to the YAML schema file.
        output_yaml_path (str): Path where the output YAML file will be written.

    Returns:
        None
    
    Raises:
        Exception: If an error occurs during validation or instance loading.
    """
    # Load the minimal JSON instance (containing only modellingAndValidation)
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    data = odict_to_dict(data)

    # Build the nested instance structure.
    instance_data = build_minimal_epd_structure(data)

    # Dump to YAML
    with open(output_yaml_path, "w", encoding="utf-8") as out:
        yaml.dump(instance_data, out, sort_keys=False, allow_unicode=True)
    print(f"modellingAndValidation EPD YAML instance written to: {output_yaml_path}")

    # Validate using the Validator
    try:
        validator = Validator(schema_path, strict=False)
        report = validator.validate(instance_data, "ModellingAndValidation")
        if report.results:
            print("Validation errors:")
            for result in report.results:
                print(result.message)
        else:
            print("The ModellingAndValidation instance is valid!")
    except Exception as e:
        print("Exception during validation:")
        print(e)
        raise

    # Load the YAML instance using YAMLLoader
    try:
        loader = YAMLLoader()
        # If the instance is wrapped in a "modellingAndValidation" key, unwrap it.
        if "modellingAndValidation" in instance_data:
            instance_data = instance_data["modellingAndValidation"]
        loader.load(instance_data, target_class=ModellingAndValidation)
        print("Instance loaded successfully!")
    except Exception as e:
        print("Exception during instance loading via YAMLLoader:")
        print(e)
        raise


# Usage
input_file = "../linkml/data/json/instance_modellingAndValidation.json"
schema_file = "../linkml/data/yaml/linkml_modellingAndValidation_schema.yaml"
output_yaml = "../linkml/data/yaml/linkml_modellingAndValidation_instance.yaml"

process_instance(input_file, schema_file, output_yaml)

modellingAndValidation EPD YAML instance written to: ../linkml/data/yaml/linkml_modellingAndValidation_instance.yaml
The ModellingAndValidation instance is valid!
Instance loaded successfully!


In [4]:
# Loading and RDF (JSON-LD, Turtle) Generation modellingAndValidation EPD YAML Instance

import yaml
from linkml_runtime.dumpers import RDFLibDumper
from linkml_runtime.loaders import YAMLLoader
from linkml_runtime.utils.schemaview import SchemaView
from data.py.linkml_modellingAndValidation_schema import ModellingAndValidation


def generate_rdf(schema_path: str, instance_path: str, output_jsonld_path: str, output_turtle_path: str) -> None:
    """
    Load a validated LinkML YAML instance, create a SchemaView, and dump the instance to RDF in JSON-LD and Turtle formats.

    Steps:
        1. Load the instance from a YAML file.
        2. Create a SchemaView from the schema file.
        3. Generate RDF in JSON-LD and Turtle formats.
        4. Write the RDF to files.
    
    Parameters:
        schema_path (str): Path to the LinkML YAML schema file.
        instance_path (str): Path to the validated LinkML YAML instance file.
        output_jsonld_path (str): Path where the output JSON-LD file will be written.
        output_turtle_path (str): Path where the output Turtle file will be written.
    
    Returns:
        None

    Raises:
        Exception: If an error occurs during instance loading, SchemaView creation, or RDF generation.
    """
    # Load instance from file
    try:
        with open(instance_path, "r", encoding="utf-8") as f:
            loaded_data = yaml.safe_load(f)  # This is now a Python dict
        # Extract "modellingAndValidation"
        if "modellingAndValidation" in loaded_data:
            loaded_data = loaded_data["modellingAndValidation"]
        # Feed loaded_data dictionary into the loader
        instance_obj = YAMLLoader().load(loaded_data, target_class=ModellingAndValidation)
        print(f"Instance loaded from: {instance_path}")
    except Exception as e:
        print("Exception during instance loading:")
        print(e)
        raise

    # Create SchemaView
    try:
        sv = SchemaView(schema_path)
        print("SchemaView created successfully.")
    except Exception as e:
        print("Exception during SchemaView creation:")
        print(e)
        raise

    # Generate and write RDF (JSON-LD, Turtle)
    try:
        dumper = RDFLibDumper()
        # JSON‑LD format
        rdf_jsonld = dumper.dumps(instance_obj, schemaview=sv, fmt="json-ld")
        with open(output_jsonld_path, "w", encoding="utf-8") as rdf_file:
            rdf_file.write(rdf_jsonld)
        print(f"RDF (JSON‑LD) successfully written to: {output_jsonld_path}")
        # Turtle format
        rdf_turtle = dumper.dumps(instance_obj, schemaview=sv, fmt="turtle")
        with open(output_turtle_path, "w", encoding="utf-8") as ttl_file:
            ttl_file.write(rdf_turtle)
        print(f"RDF (Turtle) successfully written to: {output_turtle_path}")
    except Exception as e:
        print("Exception during RDF generation:")
        print(e)
        raise


# Example usage:
schema_path = "../linkml/data/yaml/linkml_modellingAndValidation_schema.yaml"
instance_path = "../linkml/data/yaml/linkml_modellingAndValidation_instance.yaml"
output_jsonld_path = "../linkml/data/rdf/linkml_modellingAndValidation_instance.jsonld"
output_turtle_path = "../linkml/data/rdf/linkml_modellingAndValidation_instance.ttl"

generate_rdf(schema_path, instance_path, output_jsonld_path, output_turtle_path)

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Instance loaded from: ../linkml/data/yaml/linkml_modellingAndValidation_instance.yaml
SchemaView created successfully.
RDF (JSON‑LD) successfully written to: ../linkml/data/rdf/linkml_modellingAndValidation_instance.jsonld
RDF (Turtle) successfully written to: ../linkml/data/rdf/linkml_modellingAndValidation_instance.ttl
